In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX


In [2]:
# Cargar los DataFrames desde archivos Excel
path_df_location = "../dataset/df_historic_order_demand.xlsx"
df_historic_order_demand = pd.read_excel(path_df_location)

path_df_location = "../dataset/df_location.xlsx"
df_location = pd.read_excel(path_df_location)

path_df_vehicle = "../dataset/df_vehicle.xlsx"
df_vehicle = pd.read_excel(path_df_vehicle)

path_df_orders = "../dataset/df_orders.xlsx"
df_orders = pd.read_excel(path_df_orders)

path_df_distance_km = "../dataset/df_distance_km.xlsx"
df_distance_km = pd.read_excel(path_df_distance_km)

In [8]:
df_orders.head()

,cliente,mes_anio,order_demand
0,Cliente_1,12-2024,909
1,Cliente_2,12-2024,959
2,Cliente_3,12-2024,960
3,Cliente_4,12-2024,980
4,Cliente_5,12-2024,979


In [9]:
# Explorar la estructura del DataFrame
print("Primeras filas del DataFrame:")
print(df_historic_order_demand.head())

print("\nInformación del DataFrame:")
print(df_historic_order_demand.info())

# Verificar si hay valores nulos
print("\nValores nulos por columna:")
print(df_historic_order_demand.isnull().sum())


Primeras filas del DataFrame:
     cliente mes_anio  order_demand
0  Cliente_1  12-2020         857.0
1  Cliente_2  12-2020         941.0
2  Cliente_3  12-2020         878.0
3  Cliente_4  12-2020         949.0
4  Cliente_5  12-2020         932.0

Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 980 entries, 0 to 979
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   cliente       980 non-null    object 
 1   mes_anio      980 non-null    object 
 2   order_demand  932 non-null    float64
dtypes: float64(1), object(2)
memory usage: 23.1+ KB
None

Valores nulos por columna:
cliente          0
mes_anio         0
order_demand    48
dtype: int64


In [10]:
# Crear una nueva columna 'mes_anio_dia' con los datos en formato datetime
df_historic_order_demand['mes_anio_dia'] = pd.to_datetime(df_historic_order_demand['mes_anio'], format='%m-%Y', errors='coerce')

# Verificar si hay valores nulos en la nueva columna
print("\nValores nulos en 'mes_anio_dia':")
print(df_historic_order_demand['mes_anio_dia'].isnull().sum())


Valores nulos en 'mes_anio_dia':
0


In [11]:
# Verificar si hay duplicados basados en 'cliente' y 'mes_anio'
print("Registros duplicados:")
print(df_historic_order_demand.duplicated(subset=['cliente', 'mes_anio_dia']).sum())

Registros duplicados:
0


In [12]:
# Imprimir una muestra para verificar el resultado
print("Datos preprocesados:")
print(df_historic_order_demand.head())

Datos preprocesados:
     cliente mes_anio  order_demand mes_anio_dia
0  Cliente_1  12-2020         857.0   2020-12-01
1  Cliente_2  12-2020         941.0   2020-12-01
2  Cliente_3  12-2020         878.0   2020-12-01
3  Cliente_4  12-2020         949.0   2020-12-01
4  Cliente_5  12-2020         932.0   2020-12-01


In [13]:
# Ordenar primero por cliente y luego por mes_anio_dia
df_historic_order_demand = df_historic_order_demand.sort_values(by=['cliente', 'mes_anio_dia']).reset_index(drop=True)

# Verificar la estructura después de ordenar
print(df_historic_order_demand.head())  # Muestra las primeras filas para confirmar

     cliente mes_anio  order_demand mes_anio_dia
0  Cliente_1  12-2020         857.0   2020-12-01
1  Cliente_1  01-2021         768.0   2021-01-01
2  Cliente_1  02-2021         635.0   2021-02-01
3  Cliente_1  03-2021         631.0   2021-03-01
4  Cliente_1  04-2021         589.0   2021-04-01


In [14]:
# Mostrar número de valores nulos en la columna 'order_demand'
print("\nValores nulos en 'order_demand':")
print(df_historic_order_demand['order_demand'].isnull().sum())

# Mostrar las filas que contienen valores nulos en 'order_demand'
print("\nFilas con valores nulos en 'order_demand':")
print(df_historic_order_demand[df_historic_order_demand['order_demand'].isnull()])



Valores nulos en 'order_demand':
48

Filas con valores nulos en 'order_demand':
        cliente mes_anio  order_demand mes_anio_dia
55   Cliente_10  06-2021           NaN   2021-06-01
65   Cliente_10  04-2022           NaN   2022-04-01
89   Cliente_10  04-2024           NaN   2024-04-01
92   Cliente_10  07-2024           NaN   2024-07-01
109  Cliente_11  11-2021           NaN   2021-11-01
124  Cliente_11  02-2023           NaN   2023-02-01
128  Cliente_11  06-2023           NaN   2023-06-01
140  Cliente_11  06-2024           NaN   2024-06-01
169  Cliente_12  10-2022           NaN   2022-10-01
173  Cliente_12  02-2023           NaN   2023-02-01
205  Cliente_13  09-2021           NaN   2021-09-01
211  Cliente_13  03-2022           NaN   2022-03-01
229  Cliente_13  09-2023           NaN   2023-09-01
234  Cliente_13  02-2024           NaN   2024-02-01
253  Cliente_14  08-2021           NaN   2021-08-01
254  Cliente_14  09-2021           NaN   2021-09-01
275  Cliente_14  06-2023           

In [15]:
# Sustituir los valores nulos en 'order_demand' por ceros
df_historic_order_demand['order_demand'] = df_historic_order_demand['order_demand'].fillna(0)

# Verificar si quedan valores nulos
print("\nVerificación de valores nulos después de la sustitución:")
print(df_historic_order_demand['order_demand'].isnull().sum())



Verificación de valores nulos después de la sustitución:
0


In [16]:
df_historic_order_demand

,cliente,mes_anio,order_demand,mes_anio_dia
0,Cliente_1,12-2020,857.0,2020-12-01
1,Cliente_1,01-2021,768.0,2021-01-01
2,Cliente_1,02-2021,635.0,2021-02-01
3,Cliente_1,03-2021,631.0,2021-03-01
4,Cliente_1,04-2021,589.0,2021-04-01
...,...,...,...,...
975,Cliente_9,08-2024,662.0,2024-08-01
976,Cliente_9,09-2024,637.0,2024-09-01
977,Cliente_9,10-2024,627.0,2024-10-01
978,Cliente_9,11-2024,613.0,2024-11-01
